# Setting Jubilee Tool Parking Positions
This notebook scaffolds the steps outlined [here](https://www.jubilee3d.com/index.php?title=Setting_Tool_Parking_Positions) to set tool parking positions from the Jubilee wiki. Some of these steps are manual, and others require sending commands. This notebook will walk through all necessary steps!

### 0. Notebook Setup
We'll need to send commands to the machine-- the following cells set up this communication. Be sure to have a usb cable connecting your duet board to your machine,

In [4]:
# import required modules
import os
import sys
from jinja2 import Environment, FileSystemLoader, select_autoescape

sys.path.append('..')
from utils.MachineUtils import *

In [2]:
# Setup your machine connection
# List available ports in thie cell

ports = serial.tools.list_ports.comports()
print([port.name for port in ports]) 

['ttyACM1', 'ttyAMA0']


In [4]:
# Choose the correct port from above and establish connection with machine
port = '/dev/ttyACM1'
m = MachineCommunication(port)

### 1. Home your machine
Make sure the build plate is clear!

In [6]:
m.homeAll()

### 2. Coursely place your parking post.
Take your parking post assembly and loosely fasten it to the rail approximately where you want it; you should still be able to move the parking post around to adjust it for X-axis position and parking post height.

## 3. Move the build plate
Move your build plate down in the Z-axis (positive Z values) such that any tools you’re about to mount will fully clear the bed. Usually a value of z=50 or more is enough for this-- you can edit the value in the cell below to accommodate longer tools.

In [5]:
m.moveTo(z=100)

### 4. Jog the carriage to somewhere close to the center of the printer bed.

In [6]:
m.moveTo(x=150, y=150)

### 5. With no active tool, manually place your tool against the lock carriage, and lock the tool onto the carriage using the tool lock macro (Do not select a tool using the Tx commands - this is critical)
Run the following tool lock cell once you have placed the tool against the lock carriage.

In [7]:
m.tool_lock()

### 6. Jog the carriage to roughly where you want to end up across the X-axis. Eyeball it, or use a measuring tape/ruler if you want to.
You can send coordinates below, otherwise the duet3 dashboard is useful for this step.

In [ ]:
#m.moveTo(x=<approx-position>)

### 6. Jog the carriage forward in the Y-axis until its just in front of the parking post assembly, making sure you don’t engage the parking dowel pins just yet.
Again, controls from the duet console are useful here

In [ ]:
#m.moveTo(y=<approx-position>)

### 8. Line up the parking post on the rail to the tool across the X-axis.
You can do this by physically moving the parking post along the rail

### 9. Slowly jog the carriage forward in the Y-axis until the tool starts engaging the parking dowel pins.
You can do this by incrementing by y=+1 from the duet3 control dashboard.

### 10. Take a look to make sure everything is lined up: dowel pins, parking wings, etc.

### 11. Jog the carriage further in the Y-axis in small (1mm) increments until you’ve achieved a full dock. Depending on your tool mount, you may or may not have the tool actually touching the front of the extrusion that holds the parking posts.
You can do this by incrementing by y=+1 from the duet3 dashboard

### 12. Adjust the position in the X-axis to your liking (not by hand, we don’t want to lose position).
You can do this by incrementing by x=+/-0.1 from the duet3 dashboard

### 13. Once your tool and parking post are fully engaged and in the final position you want them to be, take a note of the X and Y values reported for the printer’s current position in the web interface. This will be needed for the tpost and tfree macros for this specific tool and parking position.

In [9]:
pos = m.getPosition()
toolX = pos['X']
toolY = pos['Y']
toolX, toolY

('286.000', '329.000')

### 14. Tighten down the tee-nuts to lock in your X-axis position for this tool.

### 15. Adjust your parking post height adjustment until you’re satisfied that the tool is properly supported and parallel to the Y-axis rails of your printer. Tighten down the screws for the height adjustment to lock it in.

### 16. Unlock the tool from the carriage using the tool unlock macro (and not using a T-1 or Tx command).

In [10]:
m.tool_unlock()

### 17. Jog the now-decoupled carriage backwards in the Y-axis direction until it fully clears your tool(s) and parking post(s). Take a note of this value of the Y-axis position from the web interface, as this will be close to your final choice of “Tool Clearance Limit” for all of your tfree files.
Duet3 console controls can be used here; you can run the following cell once the carriage is clear

In [10]:
yClearPos = m.getPosition()['Y']
yClearPos

'237.000'

### 18. Update the relevant tpost and tfree files for the tool you have just set up. 
There are 3 files which need to be updated with the values you've found. We'll use template files to make this quicker and more reliable-- run the following cell to set up the templating engine. 

In [11]:
env = Environment(loader= FileSystemLoader("templates"))

The "Txpost.g" file is called for tool number 'x' after the firmware thinks Toolx is selected. From the duet console, navigate to system-->Txpost.g, where x is the number of the tool you are currently calibrating-- create the file if it doesn't yet exist. Run the following cell and replace the contents (i.e. copy-paste) with the output produced:

In [17]:
template = env.get_template("tpost.g")
tpost_content = template.render(tool_number=tool_number, x_park=x_park, 
                          y_park=y_park, y_clear=y_clear, manhattan_offset = manhattan_offset)

print(tpost_content)

; tpost100.g
; called after firmware thinks Tool100 is selected
; Note: tool offsets are applied at this point!
; Note that commands preempted with G53 will NOT apply the tool offset.

; M116 P1                    ; Wait for set temperatures to be reached
; M302 P1                    ; Prevent Cold Extrudes, just in case temp setpoints are at 0

G90                        ; Ensure the machine is in absolute mode before issuing movements.

G53 G1 X50 F6000           ; Move to the pickup position with tool-1.
G53 G1 Y330 F6000
M98 P"/macros/tool_lock.g" ; Lock the tool

G1 R2 Z0                   ; Restore prior Z position before tool change was initiated.
                           ; Note: tool tip position is automatically saved to slot 2 upon the start of a tool change.
                           ; Restore Z first so we don't crash the tool on retraction.
G1 R0 Y0                   ; Retract tool by restoring Y position next now accounting for new tool offset.
                        

Similarly, tfreex.g uns at the start of a toolchange if the current tool is tool-x. Again, copy-paste the following output into tfreex.g:

In [19]:
template = env.get_template("tfree.g")
tfree_content = template.render(tool_number=tool_number, x_park=x_park, 
                          y_park=y_park, y_clear=y_clear, manhattan_offset = manhattan_offset)

print(tfree_content)

; tfree100.g
; Runs at the start of a toolchange if the current tool is tool-100.
; Note: tool offsets are applied at this point unless we preempt commands with G53!

G91                          ; Relative Mode.
G1 Z2                        ; Pop Z up slightly so we don't crash while traveling over the usable bed region.
G90                          ; Absolute Mode.

G53 G0 X50 Y305 F12000    ; Rapid to the back of the post. Stay away from the tool rack so we don't collide with tools.
                             ; This position must be chosen such that the most protruding y face of the current tool
                             ; (while on the carriage) does not collide with the most protruding y face of any parked tool.

G53 G1 Y330 F6000            ; Controlled move to the park position with tool-1. (park_x, park_y)
M98 P"/macros/tool_unlock.g" ; Unlock the tool
G53 G1 Y305 F6000            ; Retract the pin.


Finally, the tprex.g file is where the carriage will move to immediately before & after picking up a tool. It is recommended that all pick-up/drop-off moves are 'Manhattan' style (a big negative move in the Y-axis until you’re completely clear of any tools and parking posts, then a move in the X-axis) to avoid collisions with other tools or parking posts). We use a default value of 60mm. Replace tpre.x with the following contents:

In [20]:
template = env.get_template("tpre.g")
tpre_content = template.render(tool_number=tool_number, x_park=x_park, 
                          y_park=y_park, y_clear=y_clear, manhattan_offset = manhattan_offset)

print(tpre_content)

; tpre100.g
; Runs after freeing the previous tool if the next tool is tool-100.
; Note: tool offsets are not applied at this point!

G90                   ; Ensure the machine is in absolute mode before issuing movements.
G0 X110 Y305 F20000 ; Rapid to the approach position without any current tool.
G60 S0                ; Save this position as the reference point from which to later apply new tool offsets.


In [8]:
# try the template engine to do this
tool_number = 100
x_park = 50
y_park = 330
y_clear = 305
manhattan_offset = 60

In [9]:
env = Environment(loader= FileSystemLoader("templates"))
template = env.get_template("tpre.g")
content = template.render(tool_number=tool_number, x_park=x_park, 
                          y_park=y_park, y_clear=y_clear, manhattan_offset = manhattan_offset)

print(content)

; tpre100.g
; Runs after freeing the previous tool if the next tool is tool-100.
; Note: tool offsets are not applied at this point!

G90                   ; Ensure the machine is in absolute mode before issuing movements.
G0 X110 Y305 F20000 ; Rapid to the approach position without any current tool.
G60 S0                ; Save this position as the reference point from which to later apply new tool offsets.


### 19. Try out a tool change to the new parking post using the Tx command!
Test both selecting a tool and parking it. While testing, check to see if your setup needs a few tweaks to make sure the lock pin fully engages the ramp (change the Y-axis value used in tpost - higher Y values move the locking carriage closer to the tool dock) upon locking. Check to see if the tool has been docked correctly after unlocking (change the Y-axis values in tfree to adjust this to your needs).

In [34]:
toolNumber = 1 # Change this to match the tool you've just set up
m.toolChange(toolNumber)